In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import import_ipynb
import utils

importing Jupyter notebook from utils.ipynb


In [2]:
train_path, test_path = utils.load_dataset('cell_images')

## Find the average dimension of images

In [3]:
dim1_uninfected, dim2_uninfected = utils.check_image_dimensions(test_path+'uninfected/')

In [4]:
dim1_parasitized, dim2_parasitized = utils.check_image_dimensions(test_path+'parasitized/')

In [8]:
print(np.mean(dim1_parasitized))
print(np.mean(dim2_parasitized))
print(np.mean(dim1_parasitized))
print(np.mean(dim2_parasitized))

133.31846153846155